In [4]:
import requests
import pandas as pd
from datetime import datetime
import time
from typing import List, Dict
import logging

class PBPStatsAPI:
    def __init__(self, start_year: int = 2013, end_year: int = 2024):
        self.base_url = "https://api.pbpstats.com/get-game-logs/nba?"
        self.season_types = ["Regular Season", "Playoffs"]
        self.start_year = start_year
        self.end_year = end_year

    def get_season_years(self) -> List[str]:
        """Generate season strings from start_year to end_year."""
        return [f"{year}-{str(year + 1)[2:]}" for year in range(self.start_year, self.end_year + 1)]
    
    def get_team_game_logs(self, team_id: str) -> pd.DataFrame:
        """
        Fetch all game logs for a specific team within the specified year range.
        
        Args:
            team_id (str): The team ID in PBP Stats format
            
        Returns:
            pd.DataFrame: DataFrame containing all game logs
        """
        all_games = []
        seasons = self.get_season_years()
        
        for season in seasons:
            for season_type in self.season_types:
                if season_type != 'Playoffs':
                    try:
                        params = {
                            "Season": season,
                            "SeasonType": season_type,
                            "EntityId": team_id,
                            "EntityType": "Team"
                        }
                        
                        response = requests.get(self.base_url, params=params)
                        data_response = response.json()
                        raw_data = data_response["multi_row_table_data"]
                        games_data = pd.DataFrame(raw_data)
                        
                        games_data['Season'] = season
                        games_data['year'] = int(season.split('-')[0]) + 1
                        games_data['SeasonType'] = season_type
                        all_games.append(games_data)
                                
                        # Respect API rate limits
                        time.sleep(2)
                        
                    except requests.exceptions.RequestException as e:
                        logging.error(f"Error fetching data for {team_id} in {season} {season_type}: {str(e)}")
                        continue
                    
        return pd.concat(all_games) if all_games else pd.DataFrame()

def fetch_all_teams_game_logs(team_ids: List[str], start_year: int, end_year: int) -> Dict[str, pd.DataFrame]:
    """
    Fetch game logs for multiple teams within the specified year range.
    
    Args:
        team_ids (List[str]): List of team IDs
        start_year (int): Starting year for data collection
        end_year (int): Ending year for data collection
        
    Returns:
        Dict[str, pd.DataFrame]: Dictionary with team IDs as keys and their game logs as values
    """
    api = PBPStatsAPI(start_year=start_year, end_year=end_year)
    team_games = {}
    
    for team_id in team_ids:
        logging.info(f"Fetching game logs for team {team_id}")
        team_games[team_id] = api.get_team_game_logs(team_id)
        
    return team_games

# Example usage
if __name__ == "__main__":
    # Example team IDs (you'll need to use the correct IDs from PBP Stats)
    df = pd.read_csv('index_master.csv')
    df = df[df.year > 2013]
    df = df[df.team != 'TOT']
    team_ids = df['team_id'].unique().tolist()

    # Input start and end years
    start_year = 2013
    end_year =2014

    # Configure logging
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s'
    )
    
    # Fetch game logs
    team_game_logs = fetch_all_teams_game_logs(team_ids, start_year, end_year)
    
    # Example: Save to CSV files
    for team_id, games_df in team_game_logs.items():
        if not games_df.empty:
            for year in range(start_year, end_year + 1):
                team_df = games_df[games_df.year == year]
                if not team_df.empty:
                    team_df.to_csv(f"team/{year}/{team_id}.csv", index=False)
            logging.info(f"Saved game logs for {team_id}")


2024-12-11 19:58:51,055 - INFO - Fetching game logs for team 1610612761
2024-12-11 19:59:01,670 - INFO - Fetching game logs for team 1610612758
2024-12-11 19:59:09,254 - INFO - Fetching game logs for team 1610612760
2024-12-11 19:59:17,283 - INFO - Fetching game logs for team 1610612766
2024-12-11 19:59:24,733 - INFO - Fetching game logs for team 1610612749
2024-12-11 19:59:32,723 - INFO - Fetching game logs for team 1610612753
2024-12-11 19:59:41,942 - INFO - Fetching game logs for team 1610612740
2024-12-11 19:59:49,471 - INFO - Fetching game logs for team 1610612752
2024-12-11 19:59:56,909 - INFO - Fetching game logs for team 1610612757
2024-12-11 20:00:05,473 - INFO - Fetching game logs for team 1610612755
2024-12-11 20:00:12,775 - INFO - Fetching game logs for team 1610612754
2024-12-11 20:00:20,426 - INFO - Fetching game logs for team 1610612748
2024-12-11 20:00:28,896 - INFO - Fetching game logs for team 1610612763
2024-12-11 20:00:36,981 - INFO - Fetching game logs for team 161

In [3]:
for team_id, games_df in team_game_logs.items():
    
    if not games_df.empty:
        for year in range(2014,2026):
            team_df=games_df[games_df.year==year]
            team_df.to_csv(f"team/{year}/{team_id}.csv", index=False)
        logging.info(f"Saved game logs for {team_id}")

2024-12-11 19:54:32,453 - INFO - Saved game logs for 1610612761
2024-12-11 19:54:32,522 - INFO - Saved game logs for 1610612758
2024-12-11 19:54:32,592 - INFO - Saved game logs for 1610612760
2024-12-11 19:54:32,659 - INFO - Saved game logs for 1610612766
2024-12-11 19:54:32,728 - INFO - Saved game logs for 1610612749
2024-12-11 19:54:32,790 - INFO - Saved game logs for 1610612753
2024-12-11 19:54:32,859 - INFO - Saved game logs for 1610612740
2024-12-11 19:54:32,924 - INFO - Saved game logs for 1610612752
2024-12-11 19:54:32,997 - INFO - Saved game logs for 1610612757
2024-12-11 19:54:33,073 - INFO - Saved game logs for 1610612755
2024-12-11 19:54:33,136 - INFO - Saved game logs for 1610612754
2024-12-11 19:54:33,205 - INFO - Saved game logs for 1610612748
2024-12-11 19:54:33,273 - INFO - Saved game logs for 1610612763
2024-12-11 19:54:33,336 - INFO - Saved game logs for 1610612741
2024-12-11 19:54:33,399 - INFO - Saved game logs for 1610612751
2024-12-11 19:54:33,459 - INFO - Saved g